In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
def processar_vendas_excel(caminho_arquivo, nome_aba="Sheet1"):
    """
    Processa um arquivo Excel contendo dados de vendas.
    - Corrige formato de data automaticamente para evitar inversões (DD/MM/YYYY vs YYYY-DD-MM).
    - Identifica corretamente a linha do produto e associa às vendas anteriores.
    - Remove linhas de 'Total Produto'.
    - Separa código do produto e descrição corretamente.
    - Converte colunas para tipos adequados.
    - Calcula corretamente o preço unitário com base no valor líquido.

    Retorna:
    - DataFrame processado com colunas organizadas.
    """

    # ✅ **Carregar o arquivo Excel**
    df_vendas = pd.read_excel(caminho_arquivo, sheet_name=nome_aba, dtype=str)

    # ✅ **Criar coluna para armazenar os produtos corretamente**
    df_vendas["Produto"] = None
    produto_atual = None
    linhas_para_atualizar = []  # Armazena índices das linhas de venda

    for index, row in df_vendas.iterrows():
        descricao = str(row["Núm. Venda"])

        # Se encontramos um "Total Produto", pegamos o nome do produto
        if "Total Produto" in descricao:
            partes = descricao.split("Total Produto:")
            if len(partes) > 1:
                produto_atual = partes[1].strip()  # Nome correto do produto

            # Agora aplicamos esse produto a todas as linhas anteriores que precisam dele
            for i in linhas_para_atualizar:
                df_vendas.at[i, "Produto"] = produto_atual

            # Resetamos a lista, pois um novo produto será associado a novas linhas
            linhas_para_atualizar = []

        else:
            # Armazena o índice da linha que pertence ao produto atual
            linhas_para_atualizar.append(index)

    # ✅ **Remover linhas de "Total Produto" (que não são vendas)**
    df_vendas = df_vendas.dropna(subset=["Produto"])

    # ✅ **Separar Código do Produto e Descrição do Produto corretamente**
    df_vendas[["codigo", "produto"]] = df_vendas["Produto"].str.split(" ", n=1, expand=True)

    # ✅ **Remover a coluna antiga "Produto"**
    df_vendas = df_vendas.drop(columns=["Produto"])

    # ✅ **Selecionar e renomear colunas relevantes**
    df_vendas = df_vendas[[
        "Emissão", "Núm. Venda", "codigo", "produto", "Loja",
        "Qtd. Vend.", "Vlr.Total"
    ]].copy()

    # ✅ **Renomear colunas para o formato final**
    df_vendas.columns = [
        "data", "num_venda", "codigo", "produto", "loja",
        "qtd_vendida", "valor_total"
    ]

    # ✅ **Converter as datas corretamente para o formato SQLite (YYYY-MM-DD)**
    df_vendas["data"] = pd.to_datetime(df_vendas["data"], format="%d/%m/%y", errors="coerce").dt.strftime("%Y-%m-%d")

    # ✅ **Converter colunas numéricas**
    df_vendas["num_venda"] = pd.to_numeric(df_vendas["num_venda"], errors="coerce").fillna(0).astype(int)
    df_vendas["codigo"] = pd.to_numeric(df_vendas["codigo"], errors="coerce").fillna(0).astype(int)
    df_vendas["qtd_vendida"] = pd.to_numeric(df_vendas["qtd_vendida"], errors="coerce")
    df_vendas["valor_total"] = pd.to_numeric(df_vendas["valor_total"].astype(str).str.replace(',', '.'), errors="coerce")

    # ✅ **Calcular o preço unitário corretamente**
    df_vendas["preco_unit"] = np.where(
            df_vendas["qtd_vendida"] == 1,
            abs(df_vendas["valor_total"]),  # Se for 1 unidade, o preço unitário é igual ao valor total
            abs(df_vendas["valor_total"] / df_vendas["qtd_vendida"])  # Caso contrário, divide o total pela quantidade vendida
        )
    return df_vendas


In [9]:
df = processar_vendas_excel('planilhas/vendas/produtos/vendas_produtos_032024.xlsx')

z:\spx\venv\lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
import sqlite3

def salvar_dataframe_no_sqlite(df_vendas, banco_dados="vendas.db"):
    """
    Salva um DataFrame no banco de dados SQLite, evitando duplicação de vendas.

    Cada venda é identificada de forma única por:
    - data
    - num_venda
    - codigo do produto

    Se um registro já existir, ele NÃO será inserido novamente.
    """

    # Conectar ao banco SQLite
    conn = sqlite3.connect(banco_dados)
    cursor = conn.cursor()

    # Criar tabela se não existir
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS vendas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            data TEXT,
            num_venda INTEGER,
            codigo TEXT,
            produto TEXT,
            loja INTEGER,
            qtd_vendida INTEGER,
            preco_unit REAL,
            valor_total REAL,
            UNIQUE (data, num_venda, codigo, loja, qtd_vendida)  -- Permite vários produtos por venda, mas impede duplicatas
        )
    ''')
    conn.commit()

    # Converter a coluna de data para string
    df_vendas["data"] = df_vendas["data"].astype(str)

    # Contador de registros inseridos
    inseridos = 0

    # Inserir dados manualmente para evitar erro de integridade
    for _, row in df_vendas.iterrows():
        cursor.execute('''
            INSERT OR IGNORE INTO vendas (data, num_venda, codigo, produto, loja, qtd_vendida, preco_unit, valor_total)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row["data"], row["num_venda"], row["codigo"], row["produto"], row["loja"], row["qtd_vendida"], row["preco_unit"], row["valor_total"]))

        # Se a venda foi inserida, o cursor.rowcount será 1
        if cursor.rowcount > 0:
            inseridos += 1

    conn.commit()
    print(f"{inseridos} novas vendas adicionadas ao banco de dados.")  # Agora conta corretamente

    # Fechar conexão com o banco
    conn.close()



In [10]:
salvar_dataframe_no_sqlite(df)

46515 novas vendas adicionadas ao banco de dados.


In [ ]:
def consultar_por_produto(produto_parcial, banco_dados="vendas.db"):
    """
    Consulta os dados de vendas filtrando por um nome parcial do produto.
    
    Parâmetros:
    - produto_parcial: Parte do nome do produto que deseja buscar.
    - banco_dados: Nome do banco SQLite.

    Retorna:
    - DataFrame com os resultados filtrados.
    """
    conn = sqlite3.connect(banco_dados)

    # Remover espaços extras e converter para maiúsculas
    produto_parcial = produto_parcial.strip().upper()

    # Consulta SQL para buscar produtos que contenham a palavra digitada
    query = "SELECT * FROM vendas WHERE UPPER(produto) LIKE ?"
    
    # Usamos o '%' antes e depois para encontrar qualquer produto que tenha essa palavra no nome
    df_vendas = pd.read_sql(query, conn, params=(f"%{produto_parcial}%",))

    conn.close()
    
    return df_vendas



In [ ]:
# Exemplo de consulta
df_filtrado = consultar_por_produto("torsilax 30")
print(df_filtrado)

          id        data  num_venda codigo         produto  loja  qtd_vendida  \
0      28298  2025-02-06       1176  11783   TORSILAX 30CP    12            3   
1      28299  2025-02-02     123336  11783   TORSILAX 30CP     5            1   
2      28300  2025-02-07      47781  11783   TORSILAX 30CP     7            1   
3      28301  2025-02-01      44089  11783   TORSILAX 30CP     9            1   
4      28302  2025-02-08      46365  11783   TORSILAX 30CP     9            1   
...      ...         ...        ...    ...             ...   ...          ...   
4884  891387  2024-03-19      12767  11783   TORSILAX 30CP     8            1   
4885  891388  2024-03-19     131501  11783   TORSILAX 30CP     1            1   
4886  891389  2024-03-13       9449  11783   TORSILAX 30CP    10            1   
4887  891390  2024-03-15      12324  11783   TORSILAX 30CP     8            1   
4888  891391  2024-03-02     129521  11783   TORSILAX 30CP     1            1   

      preco_unit  valor_tot

In [ ]:
import sqlite3
import pandas as pd

def listar_produtos_unicos(banco_dados="vendas.db"):
    """
    Lista todos os produtos únicos armazenados no banco de dados.
    """
    conn = sqlite3.connect(banco_dados)
    query = "SELECT DISTINCT produto FROM vendas ORDER BY produto"
    df_produtos = pd.read_sql(query, conn)
    conn.close()
    return df_produtos

# Listar todos os produtos para verificar os nomes corretos
df_produtos = listar_produtos_unicos()
print(df_produtos)


                              produto
0       *PROV GEL FIXA BRONZE POS SOL
1                   20 BI 335MG 10CAP
2                   20 BI 335MG 30CAP
3                    20 BI 335MG 5CAP
4        212 VIP MEN EDT 100ML ADIPEC
...                               ...
18081         ZYMAR XD 5MG/ML COL 5ML
18082      ZYPRED 3MG+10MG/ML COL 6ML
18083            ZYRTEC 10MG 12CP REV
18084              ZYXEM 5MG 10CP REV
18085           ZYXEM 5MG/ML GTS 20ML

[18086 rows x 1 columns]
